In [2]:
import numpy as np
from pyqubo import Binary
import pyqubo
import neal
import matplotlib.pyplot as plt 
from sympy import *
from datetime import datetime
import pandas as pd

## Input Data 

In [3]:
# Input data

#Assets
asset_quantity = pd.read_csv("data/sample_asset_quantity.csv", header=None)[0].to_numpy()
asset_value = pd.read_csv("data/sample_asset_value.csv", header=None)[0].to_numpy()
asset_tiers = pd.read_csv("data/sample_asset_tiers.csv", header=None)[0].to_numpy()

n_assets = len(asset_quantity)


#Accounts 
account_exposure = pd.read_csv("data/sample_account_exposure.csv", header=None)[0].to_numpy()
account_duration = pd.read_csv("data/sample_account_duration.csv", header=None)[0].to_numpy()

n_accounts = len(account_exposure)
#single limits


#single_limits = pd.read_csv("data/sample_single_limits.csv", header=None).to_numpy() #this is the one where assets-account combos that werent in the excel sheet were set to 0

#haircuts 
haircuts = pd.read_csv("data/sample_haircuts.csv", header=None).to_numpy()/100


#Updated Tier list 

cost_factor_matrix = np.zeros(shape=(n_assets, n_accounts))

for i in range(n_assets):
    for j in range(n_accounts):
        cost_factor_matrix[i,j] = abs(account_duration[j] - asset_tiers[i])


# Penalty Terms


In [118]:
# Penalty Terms
lambda_consistency1, lambda_consistency2 = 1, 1
lambda_exposure1, lambda_exposure2 =1, 1e2
lambda_cost_fn = 1e8
# lambda_single_lim1, lambda_single_lim2 = 1e3, 1e3

# Constructing the QUBO

In [119]:
binary_variables = []
n_binary = 7
max_binary_value = 2**n_binary - 1


for i in range(n_assets):
    for j in range(n_accounts):
        for k in range(n_binary):
            globals()["x" + str(i + 1) + str(j + 1) + str(k)] = Binary("x" + str(i + 1) + str(j + 1) + str(k))
            binary_variables.append(globals()["x" + str(i + 1) + str(j + 1) + str(k)])

In [120]:

cost_function = 0

for i in range(n_assets):
    for j in range(n_accounts):
        aux_term = 0
        coeff = (cost_factor_matrix[i][j])/ (max_binary_value)
        for k in range(n_binary):
            aux_term += (2 ** (k % n_binary)) * globals()["x" + str(i + 1) + str(j + 1) + str(k)] 
        cost_function += lambda_cost_fn*coeff*aux_term
        

In [121]:
con_constraint = 0

for i in range(n_assets):
    aux_term1 = 0
    aux_term2 = 0
    for j in range(n_accounts):
        for k in range(n_binary):
            aux_term1 += (2 ** (k % n_binary)) * globals()["x" + str(i + 1) + str(j + 1) + str(k)]
    aux_term2 = (aux_term1/(max_binary_value)) - 1
    con_constraint += (lambda_consistency1 * aux_term2) + (lambda_consistency2 * (aux_term2**2))


In [122]:
# exposure req
# exposure_requirements = 0
exposure_term1 = 0
exposure_term2 = 0
for j in range(n_accounts):
    aux_term2 = 0
    aux_term3 = 0
    for i in range(n_assets):
        aux_term1 = 0
        for k in range(n_binary):
            aux_term1 += (2 ** (k % n_binary)) * globals()["x" + str(i + 1) + str(j + 1) + str(k)]
        aux_term2 += (aux_term1/max_binary_value) * asset_quantity[i] * asset_value[i] * haircuts[i][j]
    aux_term3 += aux_term2 - account_exposure[j]
    exposure_term1 += -lambda_exposure1 * aux_term3
    exposure_term2 += lambda_exposure2 * (aux_term3 ** 2)
    # exposure_requirements += (-lambda_exposure1 * aux_term3) + (lambda_exposure2 * (aux_term3 ** 2))


exposure_requirements = (exposure_term1/np.mean(asset_quantity*asset_value)) + (exposure_term2/(np.mean(asset_quantity*asset_value)**2))

In [123]:
# # single_limits

# single_limits_term = 0
# for i in range(n_assets):
#     for j in range(n_accounts):
#         aux_term1 = 0
#         aux_term2 = 0
#         for k in range(n_binary):
#             aux_term1 += (2 ** (k % n_binary)) * globals()["x" + str(i + 1) + str(j + 1) + str(k)]
#         aux_term2 += (aux_term1/(max_binary_value))*asset_quantity[i] - single_limits[i][j]
#     single_limits_term += (lambda_single_lim1 * aux_term2) + (lambda_single_lim2 * (aux_term2 ** 2))


In [124]:
# Full QUBO

final_equation = cost_function + con_constraint + exposure_requirements #+ single_limits_term

# Compile QUBO
model = final_equation.compile()
# Obtain the Q matrix
qubo, offset = model.to_qubo()

# Run Solver

In [125]:
start=datetime.now()

# Use solver to find solution
bqm = model.to_bqm()
sa = neal.SimulatedAnnealingSampler()
sampleset = sa.sample(bqm, num_reads=100, num_sweeps = 1000)
decoded_samples = model.decode_sampleset(sampleset)

# Get results
best_sample = min(decoded_samples, key=lambda x: x.energy)
results = best_sample.sample
print(f"The minimum energy is: {best_sample.energy}")

The minimum energy is: -1.304700337361453


# Verify Results

In [126]:


exposure_solution = np.zeros(shape = (n_accounts))
allocation_solution = np.zeros(shape = (n_assets))

print("=====Each Individual Allocation Percentage and Verification of Single Limits=====")

for i in range(n_assets):
    asset = 0
    for j in range(n_accounts):
        decimal = 0
        for k in range(n_binary):
            decimal +=(2**k)*results.get("x" + str(i + 1) + str(j + 1) + str(k))
        asset += decimal*(100/max_binary_value)
        print(f"Assign {decimal*(100/max_binary_value):0.2f}% of asset {i+1} to account {j+1}")
        # if (decimal/max_binary_value)*asset_quantity[i] > single_limits[i][j]:
            # print(f"Single limit constraint satisfied: {((decimal/max_binary_value)*asset_quantity[i]) <= single_limits[i][j]}")
            # print(f"Quantity assigned = {(decimal/max_binary_value)*asset_quantity[i]:0.2f}, Single limit is {single_limits[i][j]}")
        exposure_solution[j] += (decimal /max_binary_value) * asset_value[i] * asset_quantity[i]*haircuts[i][j]
    allocation_solution[i] = asset


=====Each Individual Allocation Percentage and Verification of Single Limits=====
Assign 25.20% of asset 1 to account 1
Assign 19.69% of asset 1 to account 2
Assign 0.00% of asset 1 to account 3
Assign 0.00% of asset 1 to account 4
Assign 0.00% of asset 1 to account 5
Assign 5.51% of asset 2 to account 1
Assign 5.51% of asset 2 to account 2
Assign 0.79% of asset 2 to account 3
Assign 0.00% of asset 2 to account 4
Assign 7.09% of asset 2 to account 5
Assign 0.00% of asset 3 to account 1
Assign 50.39% of asset 3 to account 2
Assign 0.00% of asset 3 to account 3
Assign 0.00% of asset 3 to account 4
Assign 0.00% of asset 3 to account 5
Assign 35.43% of asset 4 to account 1
Assign 10.24% of asset 4 to account 2
Assign 0.00% of asset 4 to account 3
Assign 0.00% of asset 4 to account 4
Assign 0.00% of asset 4 to account 5
Assign 5.51% of asset 5 to account 1
Assign 11.81% of asset 5 to account 2
Assign 5.51% of asset 5 to account 3
Assign 0.79% of asset 5 to account 4
Assign 1.57% of asset 5 

In [127]:
print("====Validation for Consistency====")
for i in range(n_assets):
    print(f"{allocation_solution[i]:0.2f}% of asset {i+1} posted")

====Validation for Consistency====
44.88% of asset 1 posted
18.90% of asset 2 posted
50.39% of asset 3 posted
45.67% of asset 4 posted
25.20% of asset 5 posted
29.92% of asset 6 posted
40.94% of asset 7 posted
43.31% of asset 8 posted
40.94% of asset 9 posted
40.16% of asset 10 posted


In [128]:
print("=====Validation for Exposure======")
for i in range(n_accounts):
    print(f"${exposure_solution[i]:,} of collateral posted, ${account_exposure[i]} required")
    print(f"{(exposure_solution[i]/account_exposure[i] - 1) *100.:0.2f}%")

=====Validation for Exposure======
$828,937.2878133476 of collateral posted, $824000.0 required
0.60%
$671,310.0964198982 of collateral posted, $669292.0 required
0.30%
$552,129.2989622079 of collateral posted, $548600.0 required
0.64%
$69,111.1274428666 of collateral posted, $64844.52 required
6.58%
$475,846.7837093987 of collateral posted, $472000.0 required
0.81%


In [129]:
print(datetime.now()-start)

0:00:08.843279


# Export to .csv

In [130]:
# Saving to an array
Q_value_unbalanced_py_qubo = np.zeros(shape=(n_assets,n_accounts))

for i in range(n_assets):
    asset = 0
    for j in range(n_accounts):
        bit = ''
        decimal = 0
        for k in range(n_binary):
            bit += str(results.get("x" + str(i + 1) + str(j + 1) + str(k)))
            decimal +=(2**k)*results.get("x" + str(i + 1) + str(j + 1) + str(k))
        asset += decimal*(100/max_binary_value)
        Q_value_unbalanced_py_qubo[i][j] = decimal/max_binary_value

pprint(f"Objective value: {sum(sum(Q_value_unbalanced_py_qubo*cost_factor_matrix))}")

Objective value: 0.973228346456693


In [ ]:
"""Uncomment the last line when you are happy with the results"""

#pd.DataFrame(Q_value_unbalanced_py_qubo).to_csv("Q_value_unbalanced_py_qubo_no_single_lim1.csv", header=None, index=None)
